In [1]:
from GraphSummarizers.Coarsener.HeteroCoarsener import HeteroCoarsener
from Datasets.NodeClassification.DBLP import DBLP
from Datasets.NodeClassification.AIFB import AIFB
from Datasets.NodeClassification.TestHetero import TestHeteroSmall, TestHeteroBig
import importlib
import torch
from test_data_converter import dgl_to_pyg_input

/home/robin/uu/thesis/convolution-matching/.venvconv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
aifb = AIFB()
original_graph = aifb.load_graph()


Done loading data from cached files.


In [ ]:
coarsener = HeteroCoarsener(None,original_graph, 0.5, num_nearest_per_etype=10, num_nearest_neighbors=10,pairs_per_level=10)
coarsener.init_step()

start create H
created H 54.72033333778381
stop init costs 7.997392177581787
stop intersection 4.789174318313599


calculate H_coarsen:   0%|          | 0/146 [00:00<?, ?it/s]/home/robin/uu/thesis/convolution-matching/GraphSummarizers/Coarsener/HeteroCoarsener.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  h = ((torch.sqrt(torch.tensor(d_u)) * s_u) + (torch.sqrt(torch.tensor(d_v)) * s_v)) / (torch.sqrt(torch.tensor(d_u_v)))
calculate H_coarsen:  83%|████████▎ | 4533/5450 [00:26<00:05, 174.74it/s]

In [5]:
merge_graph

Graph(num_nodes={'Forschungsgebiete': 71, 'Forschungsgruppen': 2, 'Kooperationen': 8, 'Personen': 164, 'Projekte': 35, 'Publikationen': 1254, '_Literal': 5380},
      num_edges={('Forschungsgebiete', 'Forschungsgebiete_ontology#dealtWithIn_Projekte', 'Projekte'): 0, ('Forschungsgebiete', 'Forschungsgebiete_ontology#isWorkedOnBy_Personen', 'Personen'): 53, ('Forschungsgebiete', 'Forschungsgebiete_ontology#name__Literal', '_Literal'): 76, ('Forschungsgebiete', 'Forschungsgebiete_rdftype__Literal', '_Literal'): 0, ('Forschungsgebiete', 'Forschungsgebiete_rev-ontology#isAbout_Projekte', 'Projekte'): 0, ('Forschungsgebiete', 'Forschungsgebiete_rev-ontology#isAbout_Publikationen', 'Publikationen'): 1915, ('Forschungsgruppen', 'Forschungsgruppen_ontology#carriesOut_Projekte', 'Projekte'): 0, ('Forschungsgruppen', 'Forschungsgruppen_ontology#head_Personen', 'Personen'): 0, ('Forschungsgruppen', 'Forschungsgruppen_ontology#homepage__Literal', '_Literal'): 5, ('Forschungsgruppen', 'Forschungsgru

In [4]:
new_g

Graph(num_nodes={'Forschungsgebiete': 146, 'Forschungsgruppen': 5, 'Kooperationen': 28, 'Personen': 237, 'Projekte': 78, 'Publikationen': 1318, '_Literal': 5450},
      num_edges={('Forschungsgebiete', 'Forschungsgebiete_ontology#dealtWithIn_Projekte', 'Projekte'): 357, ('Forschungsgebiete', 'Forschungsgebiete_ontology#isWorkedOnBy_Personen', 'Personen'): 571, ('Forschungsgebiete', 'Forschungsgebiete_ontology#name__Literal', '_Literal'): 146, ('Forschungsgebiete', 'Forschungsgebiete_rdftype__Literal', '_Literal'): 129, ('Forschungsgebiete', 'Forschungsgebiete_rev-ontology#isAbout_Projekte', 'Projekte'): 357, ('Forschungsgebiete', 'Forschungsgebiete_rev-ontology#isAbout_Publikationen', 'Publikationen'): 2120, ('Forschungsgruppen', 'Forschungsgruppen_ontology#carriesOut_Projekte', 'Projekte'): 79, ('Forschungsgruppen', 'Forschungsgruppen_ontology#head_Personen', 'Personen'): 5, ('Forschungsgruppen', 'Forschungsgruppen_ontology#homepage__Literal', '_Literal'): 5, ('Forschungsgruppen', 'Fo

In [6]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data.rdf import AIFBDataset
from Models.GNNs.HGCN import RGCN

from sklearn.metrics import accuracy_score

# Load AIFB dataset
dataset = AIFBDataset()
g = merge_graph #dataset[0]

# Check node types and label node type
print("Node types:", g.ntypes)
print("Label node type:", dataset.predict_category)  # should be 'Personen'
#g = merged_graph
label_ntype = dataset.predict_category  # 'Personen'
labels = g.nodes[label_ntype].data['labels']
train_mask = g.nodes[label_ntype].data['train_mask'].bool()
num_classes = dataset.num_classes
test_mask = g.nodes[label_ntype].data['test_mask'].to(torch.bool)
# Define embedding size
embed_size = 16
#g = merged_graph
# Create trainable input embeddings for all node types
node_embeds = nn.ModuleDict({
    ntype: nn.Embedding(g.num_nodes(ntype), embed_size)
    for ntype in g.ntypes
})

# Instantiate model
model = RGCN(in_feats=embed_size, hidden_feats=32, out_feats=num_classes, etypes=g.etypes)

# Move to device if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
node_embeds = node_embeds.to(device)
labels = labels.to(device)
train_mask = train_mask.to(device)
g = g.to(device)

# Optimizer
optimizer = torch.optim.Adam(list(model.parameters()) + list(node_embeds.parameters()), lr=0.01)

# Training loop
for epoch in range(50):
    model.train()
    
    inputs = {ntype: node_embeds[ntype].weight for ntype in g.ntypes}
    logits = model(g, inputs)[label_ntype]
    
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    pred = logits.argmax(dim=1)
    acc = accuracy_score(labels[train_mask].cpu(), pred[train_mask].cpu())
    if epoch % 10 == 0:
        # Print training loss and accuracy every 10 epochs
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | Train Acc: {acc:.4f}")

test_mask = g.nodes[label_ntype].data['test_mask'].to(torch.bool)

# Evaluate
model.eval()
with torch.no_grad():
    inputs = {ntype: node_embeds[ntype].weight for ntype in g.ntypes}
    logits = model(g, inputs)[label_ntype]
    pred = logits.argmax(dim=1)

    test_acc = accuracy_score(labels[test_mask].cpu(), pred[test_mask].cpu())
    print(f"\n✅ Test Accuracy on '{label_ntype}' nodes: {test_acc:.4f}")



Done loading data from cached files.
Node types: ['Forschungsgebiete', 'Forschungsgruppen', 'Kooperationen', 'Personen', 'Projekte', 'Publikationen', '_Literal']
Label node type: Personen
Epoch 000 | Loss: 3.1304 | Train Acc: 0.1515
Epoch 010 | Loss: 0.0035 | Train Acc: 1.0000
Epoch 020 | Loss: 0.0009 | Train Acc: 1.0000
Epoch 030 | Loss: 0.0004 | Train Acc: 1.0000
Epoch 040 | Loss: 0.0002 | Train Acc: 1.0000

✅ Test Accuracy on 'Personen' nodes: 0.7692


In [7]:
test_mask = g.nodes[label_ntype].data['test_mask'].to(torch.bool)

# Evaluate
model.eval()
with torch.no_grad():
    inputs = {ntype: node_embeds[ntype].weight for ntype in g.ntypes}
    logits = model(g, inputs)[label_ntype]
    pred = logits.argmax(dim=1)

    test_acc = accuracy_score(labels[test_mask].cpu(), pred[test_mask].cpu())
    print(f"\n✅ Test Accuracy on '{label_ntype}' nodes: {test_acc:.4f}")



✅ Test Accuracy on 'Personen' nodes: 0.8889
